In [12]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import DataFrame
from matplotlib import pyplot as plt 
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from string import punctuation
from sklearn.metrics import classification_report, confusion_matrix
import spacy
import re
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from keras.models import Sequential
from sklearn.model_selection import train_test_split

class Model():

    def __init__(self,datafile="hasoc"):
      self.training = pd.read_csv(datafile+ "_train.csv")
      self.test = pd.read_csv(datafile+ "_test.csv")  
      #Recode labels for Training task_1 to numeric
      cleanup_nums = {"task_1": {"HOF": 1, "NOT": 0}}
      self.training = self.training.replace(cleanup_nums)
      #Recode labels for Test SubtaskA to numeric
      cleanup_nums2 = {"task_1": {"HOF": 1, "NOT": 0}}
      self.test = self.test.replace(cleanup_nums2)   
    
    def preprocess(self):
      def words(text, stem_words=False):
        #Remove emojis 
        emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        "]+", flags=re.UNICODE)
        text = emoji_pattern.sub(r'', text) # No emoji
        #text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE) # Remove urls
        text = re.sub(r'http\S+', '', text) # Remove urls
        # Lowering the words in Tweets
        text = text.lower()
        # Remove punctuation from Tweets
        text = ''.join([c for c in text if c not in punctuation])
        #Use filter and lambda to remove numeric digits from string
        res = "".join(filter(lambda x: not x.isdigit(), text))
        text = str(res)
        #remove leading and ending spaces
        text = text.strip()
        # Return a list of words
        return(text)

      def process(tweet_list, tweets):
          for tweet in tweets:
              tweet_list.append(words(tweet))

      self.processed_tweet1 = []
      self.processed_tweet2 = []
      process(self.processed_tweet1, self.training["text"])
      process(self.processed_tweet2, self.test["text"])

    def fit(self,):
      from pandas import DataFrame
      #training set
      self.df0 = DataFrame(self.processed_tweet1,columns=['text'])
      #test set
      self.df00 = DataFrame(self.processed_tweet2,columns=['text'])   
      # fit a tokenizer
      def create_tokenizer(lines):
          tokenizer = Tokenizer()
          tokenizer.fit_on_texts(lines)
          return tokenizer
      # calculate the maximum document length
      def max_length(lines):
          return max([len(s.split()) for s in lines])
      # encode a list of lines
      def encode_text(tokenizer, lines, length):
          # integer encode
          encoded = tokenizer.texts_to_sequences(lines)
          # pad encoded sequences
          padded = pad_sequences(encoded, maxlen=length, padding='post')
          return padded
      # define the model
      def define_model(length, vocab_size):
          # channel 1
          model = Sequential()
          model.add(Embedding(vocab_size, 100,input_shape=(length,)))
          model.add(Conv1D(filters=32, kernel_size=4, activation='relu'))
          model.add(Dropout(0.5))
          model.add(MaxPooling1D(pool_size=2))
          model.add(Flatten())
          # interpretation
          model.add(Dense(1, activation='sigmoid')) 
          # compile
          model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
          # summarize
          return model
      # Organize  dataset
      self.x_train = self.df0['text']
      self.y_train = self.training['task_1']
      self.x_test = self.df00['text']
      self.y_test = self.test['task_1']
      # create tokenizer
      tokenizer = create_tokenizer(self.x_train)
      # calculate max document length
      length = max_length(self.x_train)
      # calculate vocabulary size
      vocab_size = len(tokenizer.word_index) + 1
      # encode data
      self.trainX = encode_text(tokenizer, self.x_train, length)
      self.testX = encode_text(tokenizer, self.x_test, length)
      self.model = define_model(length, vocab_size)
      # fit model
      results = self.model.fit(self.trainX, array(self.y_train), epochs=10, batch_size=32, verbose=1,validation_data=(self.testX, array(self.y_test)))
    
    def predict(self):
      pred = self.model.predict_classes(self.testX)
      print(classification_report(self.y_test,pred,labels=[1,0]))

if __name__ == '__main__':
    model_instance = Model()
    model_instance.preprocess()
    model_instance.fit()
    model_instance.predict()

Epoch 1/10
183/183 [==============================] - 8s 37ms/step - loss: 0.6690 - accuracy: 0.6037 - val_loss: 0.6239 - val_accuracy: 0.7476
Epoch 2/10
183/183 [==============================] - 6s 35ms/step - loss: 0.5759 - accuracy: 0.6999 - val_loss: 0.5682 - val_accuracy: 0.7372
Epoch 3/10
183/183 [==============================] - 6s 35ms/step - loss: 0.2814 - accuracy: 0.9029 - val_loss: 0.6407 - val_accuracy: 0.6644
Epoch 4/10
183/183 [==============================] - 6s 35ms/step - loss: 0.1212 - accuracy: 0.9703 - val_loss: 0.6400 - val_accuracy: 0.6800
Epoch 5/10
183/183 [==============================] - 6s 35ms/step - loss: 0.0645 - accuracy: 0.9849 - val_loss: 0.6429 - val_accuracy: 0.6956
Epoch 6/10
183/183 [==============================] - 6s 35ms/step - loss: 0.0413 - accuracy: 0.9876 - val_loss: 0.7454 - val_accuracy: 0.6644
Epoch 7/10
183/183 [==============================] - 6s 35ms/step - loss: 0.0319 - accuracy: 0.9901 - val_loss: 0.7172 - val_accuracy: 0.6826

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

           1       0.41      0.55      0.47       288
           0       0.83      0.73      0.78       865

    accuracy                           0.69      1153
   macro avg       0.62      0.64      0.62      1153
weighted avg       0.72      0.69      0.70      1153

